# 국토부 아파트 매매 실거래가 API 연동   

https://www.data.go.kr/data/15058747/openapi.do

### 1. Import Packages

In [144]:
import requests
import pandas as pd
# 데이터 가공을 위해서 
from bs4 import BeautifulSoup

In [145]:
# 현재 api가 html이 아닌 xml형식으로 이루어졌기 때문에 beautifulSoap을 import하지 않는다. 

### 2. Preprocessing

매번 법정동코드를 입력안해도 pandas로 읽어서 자동으로 처리하기 

In [146]:
df_lawd_cd= pd.read_csv("법정동코드 전체자료.txt", encoding='cp949', sep='\t')

df_lawd_cd.head()
# encoding -> cp949 / utf-8 / euc-kr
# sep이 기본적으로 ,이기 때문에 \t로 다시 쓰기 

,법정동코드,법정동명,폐지여부
0,1100000000,서울특별시,존재
1,1111000000,서울특별시 종로구,존재
2,1111010100,서울특별시 종로구 청운동,존재
3,1111010200,서울특별시 종로구 신교동,존재
4,1111010300,서울특별시 종로구 궁정동,존재


1. 법정동명을 입력하기  
2. 입력받은 값으로 법정동코드 찾기  
3. 법정동코드 리턴하기(앞의 다섯자리만)  

In [147]:
name=input("법정동명을 입력하세요: (구만 입력하세요) ")

name

법정동명을 입력하세요: (구만 입력하세요) 동대문구


'동대문구'

In [148]:
# 참고하상
df_lawd_cd.loc[975,]

#iloc는 상대값 -> 첫번째값은 무조건 0번

법정동코드    1165000000
법정동명      서울특별시 서초구
폐지여부             존재
Name: 975, dtype: object

In [149]:
df_lawd_cd[df_lawd_cd["법정동명"].str.contains(name)]

# 폐지여부가 폐지가 된 동들인 값들은 삭제하기
df_lawd_cd[df_lawd_cd["법정동명"].str.contains(name) & df_lawd_cd["폐지여부"]=="존재"]

# 이 중에서 첫번째 나오는 법정동코드값을 넘기기

df_lawd_cd[(df_lawd_cd["법정동명"].str.contains(name)) & (df_lawd_cd["폐지여부"]=="존재")].head(1).iloc[0, 0]
# 첫번재 정보의 첫번째 column인 법정동코드만 나온다

df_lawd_cd[(df_lawd_cd["법정동명"].str.contains(name)) & (df_lawd_cd["폐지여부"]=="존재")].iloc[0, 0].astype(str)[0:5]
# 문자로 전환하여 문자열 자르기

'11230'

In [150]:
def find_lawd_cd(name):
    return df_lawd_cd[(df_lawd_cd["법정동명"].str.contains(name)) & (df_lawd_cd["폐지여부"]=="존재")].iloc[0, 0].astype(str)[0:5]

In [151]:
name=input("검색지역 입력(구만 입력하세요): ")
in_lawd_cd= find_lawd_cd(name)

in_lawd_cd

검색지역 입력(구만 입력하세요): 동대문구


'11230'

In [152]:
deal_day=input("원하는 날짜 입력(YYYYMM): ")
deal_day

원하는 날짜 입력(YYYYMM): 202111


'202111'

### 3. Data Request

#### Pre-definded: 

LAWD_CD와 DEAL_YMD는 수정할 수 있다.  
지역코드와 계약월을 수정하면 그게 맞는 정보를 가져올 수 있다. 


In [153]:
base_url="http://openapi.molit.go.kr:8081/OpenAPI_ToolInstallPackage/service/rest/RTMSOBJSvc/getRTMSDataSvcAptTrade?"
lawd_cd="LAWD_CD="+ in_lawd_cd +"&"
deal_ymd="DEAL_YMD="+ deal_day +"&"



In [154]:
svc_key = "serviceKey=fq448BTENSGVmu47zvzjDudVK1LUOPdIONYmOk774uKTYukGYTIZdMjVIURNLrwHFbDrRgCZRXehyB%2FR2YxQ5A%3D%3D"

In [155]:
url=base_url+lawd_cd+deal_ymd+svc_key

url

'http://openapi.molit.go.kr:8081/OpenAPI_ToolInstallPackage/service/rest/RTMSOBJSvc/getRTMSDataSvcAptTrade?LAWD_CD=11230&DEAL_YMD=202111&serviceKey=fq448BTENSGVmu47zvzjDudVK1LUOPdIONYmOk774uKTYukGYTIZdMjVIURNLrwHFbDrRgCZRXehyB%2FR2YxQ5A%3D%3D'

In [156]:
resp=requests.get(url)

resp

<Response [200]>

In [157]:
resp.content

b'<?xml version="1.0" encoding="UTF-8" standalone="yes"?><response><header><resultCode>00</resultCode><resultMsg>NORMAL SERVICE.</resultMsg></header><body><items><item><\xea\xb1\xb0\xeb\x9e\x98\xea\xb8\x88\xec\x95\xa1>    42,250</\xea\xb1\xb0\xeb\x9e\x98\xea\xb8\x88\xec\x95\xa1><\xea\xb1\xb0\xeb\x9e\x98\xec\x9c\xa0\xed\x98\x95>\xec\xa4\x91\xea\xb0\x9c\xea\xb1\xb0\xeb\x9e\x98</\xea\xb1\xb0\xeb\x9e\x98\xec\x9c\xa0\xed\x98\x95><\xea\xb1\xb4\xec\xb6\x95\xeb\x85\x84\xeb\x8f\x84>2016</\xea\xb1\xb4\xec\xb6\x95\xeb\x85\x84\xeb\x8f\x84><\xeb\x85\x84>2021</\xeb\x85\x84><\xeb\xb2\x95\xec\xa0\x95\xeb\x8f\x99> \xec\x9a\xa9\xeb\x91\x90\xeb\x8f\x99</\xeb\xb2\x95\xec\xa0\x95\xeb\x8f\x99><\xec\x95\x84\xed\x8c\x8c\xed\x8a\xb8>\xeb\x8f\x99\xeb\x8c\x80\xeb\xac\xb8\xed\x91\xb8\xeb\xa5\xb4\xec\xa7\x80\xec\x98\xa4\xec\x8b\x9c\xed\x8b\xb0(\xeb\x8f\x84\xec\x8b\x9c\xed\x98\x95\xec\x83\x9d\xed\x99\x9c\xec\xa3\xbc\xed\x83\x9d)</\xec\x95\x84\xed\x8c\x8c\xed\x8a\xb8><\xec\x9b\x94>11</\xec\x9b\x94><\xec\x9d\xbc>4</\

In [158]:
soup = BeautifulSoup(resp.content, "lxml")

# xml 파싱할 때 lxml을 사용하면 된다. 

soup

<?xml version="1.0" encoding="UTF-8" standalone="yes"?><html><body><response><header><resultcode>00</resultcode><resultmsg>NORMAL SERVICE.</resultmsg></header><items><item>    42,250거래금액&gt;중개거래거래유형&gt;2016건축년도&gt;2021년&gt; 용두동법정동&gt;동대문푸르지오시티(도시형생활주택)아파트&gt;11월&gt;4일&gt;30.11전용면적&gt;서울 동대문구중개사소재지&gt;104-3지번&gt;11230지역코드&gt;3층&gt; 해제사유발생일&gt; 해제여부&gt;</item><item>    83,000거래금액&gt;중개거래거래유형&gt;1998건축년도&gt;2021년&gt; 용두동법정동&gt;두산베어스타워아파트&gt;11월&gt;6일&gt;155.66전용면적&gt;서울 동대문구중개사소재지&gt;138-41지번&gt;11230지역코드&gt;13층&gt; 해제사유발생일&gt; 해제여부&gt;</item><item>    49,000거래금액&gt;중개거래거래유형&gt;2009건축년도&gt;2021년&gt; 용두동법정동&gt;롯데캐슬피렌체아파트&gt;11월&gt;11일&gt;30.4전용면적&gt;서울 동대문구중개사소재지&gt;790지번&gt;11230지역코드&gt;8층&gt;21.12.22해제사유발생일&gt;O해제여부&gt;</item><item>    49,000거래금액&gt;중개거래거래유형&gt;2009건축년도&gt;2021년&gt; 용두동법정동&gt;롯데캐슬피렌체아파트&gt;11월&gt;11일&gt;30.4전용면적&gt;서울 동대문구중개사소재지&gt;790지번&gt;11230지역코드&gt;8층&gt; 해제사유발생일&gt; 해제여부&gt;</item><item>    17,500거래금액&gt;중개거래거래유형&gt;2016건축년도&gt;2021년&gt; 용두동법정동&gt;동대문푸르지오시티(도시형생활주택

In [159]:
soup.find_all('item')

[<item>    42,250거래금액&gt;중개거래거래유형&gt;2016건축년도&gt;2021년&gt; 용두동법정동&gt;동대문푸르지오시티(도시형생활주택)아파트&gt;11월&gt;4일&gt;30.11전용면적&gt;서울 동대문구중개사소재지&gt;104-3지번&gt;11230지역코드&gt;3층&gt; 해제사유발생일&gt; 해제여부&gt;</item>,
 <item>    83,000거래금액&gt;중개거래거래유형&gt;1998건축년도&gt;2021년&gt; 용두동법정동&gt;두산베어스타워아파트&gt;11월&gt;6일&gt;155.66전용면적&gt;서울 동대문구중개사소재지&gt;138-41지번&gt;11230지역코드&gt;13층&gt; 해제사유발생일&gt; 해제여부&gt;</item>,
 <item>    49,000거래금액&gt;중개거래거래유형&gt;2009건축년도&gt;2021년&gt; 용두동법정동&gt;롯데캐슬피렌체아파트&gt;11월&gt;11일&gt;30.4전용면적&gt;서울 동대문구중개사소재지&gt;790지번&gt;11230지역코드&gt;8층&gt;21.12.22해제사유발생일&gt;O해제여부&gt;</item>,
 <item>    49,000거래금액&gt;중개거래거래유형&gt;2009건축년도&gt;2021년&gt; 용두동법정동&gt;롯데캐슬피렌체아파트&gt;11월&gt;11일&gt;30.4전용면적&gt;서울 동대문구중개사소재지&gt;790지번&gt;11230지역코드&gt;8층&gt; 해제사유발생일&gt; 해제여부&gt;</item>,
 <item>    17,500거래금액&gt;중개거래거래유형&gt;2016건축년도&gt;2021년&gt; 용두동법정동&gt;동대문푸르지오시티(도시형생활주택)아파트&gt;11월&gt;24일&gt;19.51전용면적&gt;서울 동대문구중개사소재지&gt;104-3지번&gt;11230지역코드&gt;8층&gt; 해제사유발생일&gt; 해제여부&gt;</item>,
 <item>    97,000거래금액&gt;중개거래거래유형&gt;1994

#### XML Parsing

In [160]:
import xml.etree.ElementTree as ET

def find_xml_items(response):
    root = ET.fromstring(response.content)
    item_list = []
    for child in root.find('body').find('items'):
        elements = child.findall('*')
        data = {}
        for element in elements:
            tag = element.tag.strip()
            text = element.text.strip()
            # print tag, text
            data[tag] = text
        item_list.append(data)  
    return item_list

In [161]:
item_list=find_xml_items(resp)

item_list 
# dictionary 형태 -> dataFrame 형태로 바꾸기 

[{'거래금액': '42,250',
  '거래유형': '중개거래',
  '건축년도': '2016',
  '년': '2021',
  '법정동': '용두동',
  '아파트': '동대문푸르지오시티(도시형생활주택)',
  '월': '11',
  '일': '4',
  '전용면적': '30.11',
  '중개사소재지': '서울 동대문구',
  '지번': '104-3',
  '지역코드': '11230',
  '층': '3',
  '해제사유발생일': '',
  '해제여부': ''},
 {'거래금액': '83,000',
  '거래유형': '중개거래',
  '건축년도': '1998',
  '년': '2021',
  '법정동': '용두동',
  '아파트': '두산베어스타워',
  '월': '11',
  '일': '6',
  '전용면적': '155.66',
  '중개사소재지': '서울 동대문구',
  '지번': '138-41',
  '지역코드': '11230',
  '층': '13',
  '해제사유발생일': '',
  '해제여부': ''},
 {'거래금액': '49,000',
  '거래유형': '중개거래',
  '건축년도': '2009',
  '년': '2021',
  '법정동': '용두동',
  '아파트': '롯데캐슬피렌체',
  '월': '11',
  '일': '11',
  '전용면적': '30.4',
  '중개사소재지': '서울 동대문구',
  '지번': '790',
  '지역코드': '11230',
  '층': '8',
  '해제사유발생일': '21.12.22',
  '해제여부': 'O'},
 {'거래금액': '49,000',
  '거래유형': '중개거래',
  '건축년도': '2009',
  '년': '2021',
  '법정동': '용두동',
  '아파트': '롯데캐슬피렌체',
  '월': '11',
  '일': '11',
  '전용면적': '30.4',
  '중개사소재지': '서울 동대문구',
  '지번': '790',
  '지역코드': '11230',
  '층': '8

In [162]:
pd.DataFrame(item_list)

,거래금액,거래유형,건축년도,년,법정동,아파트,월,일,전용면적,중개사소재지,지번,지역코드,층,해제사유발생일,해제여부
0,"42,250",중개거래,2016,2021,용두동,동대문푸르지오시티(도시형생활주택),11,4,30.11,서울 동대문구,104-3,11230,3,,
1,"83,000",중개거래,1998,2021,용두동,두산베어스타워,11,6,155.66,서울 동대문구,138-41,11230,13,,
2,"49,000",중개거래,2009,2021,용두동,롯데캐슬피렌체,11,11,30.4,서울 동대문구,790,11230,8,21.12.22,O
3,"49,000",중개거래,2009,2021,용두동,롯데캐슬피렌체,11,11,30.4,서울 동대문구,790,11230,8,,
4,"17,500",중개거래,2016,2021,용두동,동대문푸르지오시티(도시형생활주택),11,24,19.51,서울 동대문구,104-3,11230,8,,
5,"97,000",중개거래,1994,2021,용두동,신동아,11,30,84.63,서울 성동구,779,11230,9,,
6,"98,000",중개거래,2004,2021,제기동,한신아파트,11,16,84.76,서울 동대문구,1212,11230,17,,
7,"130,000",중개거래,2012,2021,전농동,래미안아름숲,11,1,84.91,서울 동대문구,688,11230,12,,
8,"110,000",중개거래,1997,2021,전농동,동아,11,13,114.81,서울 동대문구,645-2,11230,12,,
9,"160,000",중개거래,2014,2021,전농동,래미안크레시티,11,14,84.96,서울 동대문구,690,11230,14,,


### 3. Result : Save file

In [163]:
result=pd.DataFrame(item_list)

In [164]:
# output_지역_기간.csv

# f formatting으로 사용하기 

file_name= f"output_{name}_{deal_day}.csv"

In [165]:
result.to_csv(file_name, index=False, encoding='cp949')

### 필요한 정보만 발췌하기 